In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
users = pd.read_csv('users_info.csv', index_col=0)
users = users.rename(columns = {'users':'nickname'})

In [3]:
df_stocks = pd.read_csv('stocks_data/stocks_panel_data.csv')
df_stocks['Date'] = pd.to_datetime(df_stocks['Date'])
df_stocks['post utility'] = [0 for i in range(len(df_stocks))]
df_stocks

,Open,Adj Close,High,Low,Volume,Date,RSI,MACD,Symbols,post utility
0,5.1,5.534,5.663,5.1,33721900.0,2022-03-24,NaN,0.000000,CBOM,0
1,313.0,329.000,369.900,300.4,1767605.0,2022-03-24,NaN,0.000000,TATNP,0
2,1130.0,1343.600,1449.000,1130.0,1330520.0,2022-03-24,NaN,0.000000,NVTK,0
3,82.0,89.390,99.800,82.0,20195080.0,2022-03-24,NaN,0.000000,ALRS,0
4,1315.0,1280.000,1439.000,1250.0,2674028.0,2022-03-24,NaN,0.000000,CHMF,0
...,...,...,...,...,...,...,...,...,...,...
26273,2290.0,2318.400,2325.600,2282.4,141898.0,2024-03-01,62.268804,32.266148,POSI,0
26274,980.0,982.600,997.800,963.8,1824866.0,2024-03-01,80.408591,62.533227,LSRG,0
26275,3723.0,3658.500,3726.000,3645.5,160829.0,2024-03-01,44.301075,-49.208009,SMLT,0
26276,5647.0,5648.000,5668.000,5632.0,21050.0,2024-03-01,48.351648,1.104029,BELU,0


In [4]:
def calculate_post_utility(row):
    base_utility = np.sqrt(abs(row['dostoevski_score'] + row['bert_score']) / 2 * row['yields'] * row['followers'])
    if row['dostoevski_score'] + row['bert_score'] < 0:
        return -base_utility
    return base_utility

for file in os.listdir('text_data/'):
    df = pd.read_csv(f'text_data/{file}').drop(columns=['id', 'text']).rename(columns={'inserted': 'Date'})
    df['Date'] = pd.to_datetime(df['Date'])
    merged_df = df.merge(users, on='nickname', how='left')
    merged_df['yields'] = merged_df['yields'].clip(lower=0)
    merged_df['post utility'] = merged_df.apply(calculate_post_utility, axis=1)
    res = merged_df.groupby('Date')['post utility'].sum().reset_index()
    
    tick = file.split('_')[-1].split('.')[0]
    df_stocks_temp = df_stocks[df_stocks['Symbols'] == tick].copy()
    
    # Ensuring there's a direct match on 'Date' for the merge
    df_stocks_temp = df_stocks_temp.merge(res, on='Date', how='left', suffixes=('', '_updated'))
    df_stocks_temp['post utility'] = df_stocks_temp['post utility_updated'].fillna(df_stocks_temp['post utility'])
    df_stocks_temp.drop(columns=['post utility_updated'], inplace=True)
    
    # Instead of updating df_stocks directly, replace the relevant slices.
    df_stocks = pd.concat([df_stocks[df_stocks['Symbols'] != tick], df_stocks_temp])

# After the loop, you might need to sort df_stocks if the order is important
df_stocks = df_stocks.sort_values(by=['Symbols', 'Date']).reset_index(drop=True)

In [6]:
df_stocks.fillna(0, inplace=True)
df_stocks

,Open,Adj Close,High,Low,Volume,Date,RSI,MACD,Symbols,post utility
0,11.505,12.890,14.414,11.500,93857400.0,2022-03-24,0.000000,0.000000,AFKS,0.000000
1,13.100,12.603,13.299,12.603,31222800.0,2022-03-25,0.000000,-0.022895,AFKS,0.000000
2,12.661,11.650,12.698,11.400,20587500.0,2022-03-28,0.000000,-0.116594,AFKS,123.395070
3,11.650,11.540,12.400,11.300,31094600.0,2022-03-29,0.000000,-0.197451,AFKS,-75.870211
4,12.200,12.500,12.690,11.710,27744600.0,2022-03-30,0.000000,-0.181970,AFKS,-2177.606755
...,...,...,...,...,...,...,...,...,...,...
26273,3345.000,3404.800,3411.200,3294.000,1771546.0,2024-02-26,69.543369,152.537358,YNDX,-2701.576475
26274,3415.000,3365.800,3417.000,3340.000,1088468.0,2024-02-27,64.404122,146.820358,YNDX,-1322.199132
26275,3365.800,3375.800,3396.000,3337.000,960594.0,2024-02-28,53.700466,141.465780,YNDX,-1531.769590
26276,3384.000,3383.000,3400.000,3350.000,630597.0,2024-02-29,55.690575,136.232812,YNDX,-1714.027422


In [9]:
df_stocks.to_csv('stocks_data/stocks_panel_data_with_utlity.csv')